# Análisis de Ventas
Este notebook realiza un análisis de datos de ventas generados aleatoriamente para demostrar la capacidad de procesamiento de Apache Spark y la visualización con Matplotlib en Python. El objetivo es calcular y visualizar las ventas totales por categoría de producto, demostrando el flujo de trabajo desde la extracción hasta la visualización de grandes volúmenes de datos.

In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import matplotlib.pyplot as plt
import requests

# Configuración de Spark Session
spark = SparkSession.builder \
    .master("spark://aea1d7a08c36:7077") \
    .appName("SalesDataAnalysis") \
    .getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
def fetch_data():
    url = "https://random-data-api.com/api/commerce/random_commerce?size=1"
    response = requests.get(url)
    data = response.json()
    return pd.DataFrame(data)

# Traer datos múltiples veces para simular un archivo grande
df = pd.concat([fetch_data() for _ in range(10)], ignore_index=True)  # Ajusta el rango para más datos

# Convertir a Spark DataFrame
sdf = spark.createDataFrame(df)
sdf.show()

In [ ]:
from pyspark.sql.functions import col

# Suponiendo que los datos tienen una columna 'price' y 'quantity'
sales_totals = sdf.withColumn("total_sales", col("price") * col("quantity")) \
                  .groupBy("product_name") \
                  .sum("total_sales")

sales_totals.show()

In [ ]:
# Convertir a Pandas DataFrame para visualización
pd_df = sales_totals.toPandas()

# Crear gráfico
plt.figure(figsize=(10, 6))
plt.bar(pd_df['product_name'], pd_df['sum(total_sales)'])
plt.xlabel('Product Name')
plt.ylabel('Total Sales')
plt.title('Total Sales by Product Name')

# Guardar como PNG
plt.savefig('sales_by_product.png')

# Guardar como HTML
plt.savefig('sales_by_product.html')

plt.show()